<a href="https://colab.research.google.com/github/JamesTan44181/DLI-Group-Assignment/blob/main/DLI_GROUP_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>